# Imports

In [2]:
!pip install darts --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
sagemaker 2.145.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 6.1.0 which is incompatible.
sagemaker 2.145.0 requires PyYAML==5.4.1, but you have pyyaml 6.0 which is incompatible.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.


In [3]:
import os
import itertools
import boto3
import pickle

import pandas as pd
import numpy as np

from sagemaker import get_execution_role

import darts 
from darts import TimeSeries
from darts.metrics import mape, mae
from darts.models import LightGBMModel

/opt/conda/lib/python3.7/site-packages/pkg_resources/__init__.py:119: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  PkgResourcesDeprecationWarning,
/opt/conda/lib/python3.7/site-packages/pkg_resources/__init__.py:119: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  PkgResourcesDeprecationWarning,


# Global Variables

In [4]:
TRAIN_TEST_SPLIT_TIMESTAMP = pd.Timestamp("2022-01-01 00:00:00") # Test set split is Jan 1, 2022
SRC_FCST_SPLIT_TIMESTAMP = pd.Timestamp("2019-01-01 00:00:00")
LAGS = 24*14
STRIDE=24
RANDOM_STATE = 88
LAST_POINTS_ONLY = False
VERBOSE = True
LAGS_PAST_COVARIATES =  [-1, -2, -3, -24, -48, -LAGS]

# Functions

In [5]:
def make_fit_hist_fcst_lgbm_model(
    series,
    lag_params,
    optional_model_params,
    optional_covariates,
    optional_hist_fcst_params
):
    """
    Create a LightGBM time series model and fits it to the input series.
    Generates historical forecasts using the trained model and returns
    both the historical forecasts and the trained model.

    Args:
        series (Darts TimeSeries): data to train the model on.
        lag_params (dict): Parameters for the lagged features used by the model.
        optional_model_params (dict): Optional parameters for the LightGBM model.
        optional_covariates (dict): Optional additional covariates to be included in the model.
        optional_hist_fcst_params (dict): Optional parameters for generating historical forecasts.
    Returns:
        historical_fcsts (list of Darts TimeSeries): The historical forecasts generated by the model.
        model (Darts LightGBMModel): The trained LightGBM model.
    """
    # Create and fit model
    model = LightGBMModel(**lag_params, **optional_model_params)
    model.fit(train, **optional_covariates)
    
    # Create historical forecasts
    historical_fcsts = model.historical_forecasts(series, **optional_covariates, **optional_hist_fcst_params)
    
    return historical_fcsts, model

In [6]:
def calc_mape_from_hist_fcsts(val, historical_fcsts):
    """
    Calcaulates mean absolute percentage error (MAPE) from Darts historical
    forecasts, which are outputted as a list of Darts TimeSeries objects.
    
    Args:
        val (Darts TimeSeries): validation set, function will automatically calculate 
        errors between predictions and actual validation values only for timesteps in 
        which predictions exist.
        historical_fcsts (list of Darts TimeSeries): output from .historical_forecasts()
        
    Returns:
        Average MAPE (float): the average of the mean absolute percentage errors
        from each of the forecasts in historical_fcsts
    """
    return np.array([darts.metrics.metrics.mape(val, fcst) for fcst in historical_fcsts]).mean()

In [7]:
def convert_hist_fcsts_to_df(historical_fcsts, forecast_horizon):
    """
    Takes list of historical forecasts and converts them into a pandas dataframe.
    
    Args:
         historical_fcsts (list of Darts TimeSeries): output from .historical_forecasts()
         forecast_horizon: number of hours in used to make the historical forecasts
    
    Returns:
        preds_df (pandas DataFrame): a pd df of historical forecasts. Index of the df
        is a DatetimeIndex of dates like YYYY-MM-DD and each column represents
        an hour. The value at the combination of row and column is the prediction
        for that time. Example: index value of 2020-02-02 and column of 2 would
        be the prediction for 2020-02-02 02:00:00.
    """
    # Create an empty dataframe to place predictions with rows equal to val length
    # and columns equal to forecast horizon
    preds_df_index = pd.date_range(start=historical_fcsts[0].start_time().date(),
                                                              end=historical_fcsts[-1].start_time().date(),
                                                              freq='D')
    preds_df = pd.DataFrame(index=preds_df_index, columns=np.arange(0, forecast_horizon))
    preds_df.index.name = 'utc_date'
    
    for i, utc_date in enumerate(preds_df.index):
        preds_df.loc[historical_fcsts[i].time_index[0]] = historical_fcsts[i].pd_series().values.reshape(1, -1)[0]

    return preds_df

In [8]:
def calc_mape_from_df(val, preds_df, forecast_horizon):
    """
    Calcaulates mean absolute percentage error (MAPE) from pandas
    DataFrame returned by convert_hist_fcsts_to_df().
    
    Args:
        val (Darts TimeSeries): validation set, function will automatically calculate 
        errors between predictions and actual validation values only for timesteps in 
        which predictions exist.
       preds_df (pandas DataFrame): a pd df of historical forecasts.
       forecast_horizon: number of hours in used to make the historical forecasts
        
    Returns:
        Average MAPE (float): the average of the mean absolute percentage errors
        from each of the forecasts in preds_df
    """
    hours = range(forecast_horizon)
    mapes = np.zeros(preds_df.shape[0]) 

    for i in range(preds_df.shape[0]):
        date = preds_df.iloc[i].name
        times = pd.DatetimeIndex([date + pd.Timedelta(hours=hour) for hour in hours])
        preds_values = preds_df.loc[date].values
        preds_timeseries = TimeSeries.from_times_and_values(times=times, values=preds_values)
        mapes[i] = mape(val, preds_timeseries)
        
    return mapes.mean()

# Load EIA and NCAR combined dataset (preprocessed)

In [9]:
# S3 role
role = get_execution_role()
BUCKET='ucb-mids-capstone'

In [16]:
# Read in preprocessed train and validation dataset
data_key = 'Data/EIA/train_val_test_w_src_fcsts.csv'
data_location = f's3://{BUCKET}/{data_key}'
df = pd.read_csv(data_location,
                             parse_dates=True,
                              index_col='utc_time')
print("Shape is: {}".format(df.shape))

Shape is: (35160, 70)


In [17]:
# Make timeseries of entire dataset
series = TimeSeries.from_dataframe(df=df,
                                                               value_cols='co2_emissions_intensity_for_consumed_electricity')

# Make train and val
train, test = series.split_before(TRAIN_TEST_SPLIT_TIMESTAMP)

# Ablation Study

Here we keep all time-based covariates constant and examine only the change in removing a single past or future covariate at a time. We also only use a 24 hour forecast on a retrain interval of 730 hours.

In [28]:
past_covariates_cols = ['co2_emissions_consumed', 'ng_ng',  'ng_wnd', 'ng_sun', 'ng_wat', 'ng_col', 'ng_nuc', 'd'] +\
                                        ['utc_month', 'utc_week_of_year', 'utc_day_of_week', 'utc_hour']

future_covariates_cols = ['ng_ng_fcst', 'ng_col_fcst', 'ng_sun_fcst', 'ng_wat_fcst', 'ng_wnd_fcst',
                     'utc_month', 'utc_week_of_year', 'utc_day_of_week', 'utc_hour']

if past_covariates_cols:
    past_covariates = TimeSeries.from_dataframe(df=df,
                      value_cols=past_covariates_cols,
                      freq='H')
if future_covariates_cols:
    future_covariates = TimeSeries.from_dataframe(df=df,
                      value_cols=future_covariates_cols,
                      freq='H')

lag_params = {
    'lags': LAGS,
    'lags_past_covariates': LAGS_PAST_COVARIATES,
    'lags_future_covariates': [1, 2, 3, 4, 5, 6, 12, 18, 24]
}

optional_covariates = {
    'past_covariates': past_covariates,
    'future_covariates': future_covariates,
}

optional_model_params = {
    'random_state': RANDOM_STATE#,
    #'likelihood': 'quantile',    # Uncomment these for probabilistic model
    #'quantiles': [0.05, 0.5, 0.95] # Middle value must be 0.5
}

optional_samples = {
    'num_samples': None # Set this to 'None' for deterministic, 100 or other for probabalistic
}

optional_hist_fcst_params = {
    'start': TRAIN_TEST_SPLIT_TIMESTAMP,
    'forecast_horizon': 24, # only a 24 hour forecast horizon
    'stride': STRIDE,
    'retrain': 730,
    'last_points_only': LAST_POINTS_ONLY,
    'verbose': VERBOSE
}

# drop keys which are None
lag_params = {k: v for k, v in lag_params.items() if v is not None}
optional_covariates = {k: v for k, v in optional_covariates.items() if v is not None}
optional_model_params = {k: v for k, v in optional_model_params.items() if v is not None}
optional_samples = {k: v for k, v in optional_samples.items() if v is not None}
optional_hist_fcst_params = {k: v for k, v in optional_hist_fcst_params.items() if v is not None}

co2_24_historical_fcsts, co2_24_hr_model = make_fit_hist_fcst_lgbm_model(
                                                                                    series=series,
                                                                                    lag_params=lag_params,
                                                                                    optional_model_params=optional_model_params,
                                                                                    optional_covariates=optional_covariates,
                                                                                    optional_hist_fcst_params=optional_hist_fcst_params
                                                                            )

co2_24_hist_fcst_mape_all_cov = calc_mape_from_hist_fcsts(val=test, historical_fcsts=co2_24_historical_fcsts)
print(f'CO2 intensity 24 hr forecast MAPE: {round(co2_24_hist_fcst_mape_all_cov, 2)}%')

  0%|          | 0/368 [00:00<?, ?it/s]

CO2 intensity 24 hr forecast MAPE: 6.56%


## Remove one past covariate

In [38]:
past_covariates_cols = ['co2_emissions_consumed', 'ng_ng',  'ng_wnd', 'ng_sun', 'ng_wat', 'ng_col', 'ng_nuc', 'd']
past_covariates_drop_one = []

for i in range(len(past_covariates_cols)):
    past_covariates_drop_one.append(past_covariates_cols[:i] + past_covariates_cols[i+1:])

In [39]:
mapes = []

for past_covariates in past_covariates_drop_one:

    past_covariates_cols = past_covariates + ['utc_month', 'utc_week_of_year', 'utc_day_of_week', 'utc_hour']

    future_covariates_cols = ['ng_ng_fcst', 'ng_col_fcst', 'ng_sun_fcst', 'ng_wat_fcst', 'ng_wnd_fcst',
                         'utc_month', 'utc_week_of_year', 'utc_day_of_week', 'utc_hour']

    if past_covariates_cols:
        past_covariates = TimeSeries.from_dataframe(df=df,
                          value_cols=past_covariates_cols,
                          freq='H')
    if future_covariates_cols:
        future_covariates = TimeSeries.from_dataframe(df=df,
                          value_cols=future_covariates_cols,
                          freq='H')

    lag_params = {
        'lags': LAGS,
        'lags_past_covariates': LAGS_PAST_COVARIATES,
        'lags_future_covariates': [1, 2, 3, 4, 5, 6, 12, 18, 24]
    }

    optional_covariates = {
        'past_covariates': past_covariates,
        'future_covariates': future_covariates,
    }

    optional_model_params = {
        'random_state': RANDOM_STATE#,
        #'likelihood': 'quantile',    # Uncomment these for probabilistic model
        #'quantiles': [0.05, 0.5, 0.95] # Middle value must be 0.5
    }

    optional_samples = {
        'num_samples': None # Set this to 'None' for deterministic, 100 or other for probabalistic
    }

    optional_hist_fcst_params = {
        'start': TRAIN_TEST_SPLIT_TIMESTAMP,
        'forecast_horizon': 24, # only a 24 hour forecast horizon
        'stride': STRIDE,
        'retrain': 730,
        'last_points_only': LAST_POINTS_ONLY,
        'verbose': VERBOSE
    }

    # drop keys which are None
    lag_params = {k: v for k, v in lag_params.items() if v is not None}
    optional_covariates = {k: v for k, v in optional_covariates.items() if v is not None}
    optional_model_params = {k: v for k, v in optional_model_params.items() if v is not None}
    optional_samples = {k: v for k, v in optional_samples.items() if v is not None}
    optional_hist_fcst_params = {k: v for k, v in optional_hist_fcst_params.items() if v is not None}

    co2_24_historical_fcsts, co2_24_hr_model = make_fit_hist_fcst_lgbm_model(
                                                                                        series=series,
                                                                                        lag_params=lag_params,
                                                                                        optional_model_params=optional_model_params,
                                                                                        optional_covariates=optional_covariates,
                                                                                        optional_hist_fcst_params=optional_hist_fcst_params
                                                                                )
    co2_24_hist_fcst_mape = calc_mape_from_hist_fcsts(val=test, historical_fcsts=co2_24_historical_fcsts)
    print(f'CO2 intensity 24 hr forecast MAPE: {round(co2_24_hist_fcst_mape, 2)}%')
    mapes.append(co2_24_hist_fcst_mape)

  0%|          | 0/368 [00:00<?, ?it/s]

CO2 intensity 24 hr forecast MAPE: 6.52%


  0%|          | 0/368 [00:00<?, ?it/s]

CO2 intensity 24 hr forecast MAPE: 6.61%


  0%|          | 0/368 [00:00<?, ?it/s]

...

CO2 intensity 24 hr forecast MAPE: 6.67%


  0%|          | 0/368 [00:00<?, ?it/s]

CO2 intensity 24 hr forecast MAPE: 6.5%


  0%|          | 0/368 [00:00<?, ?it/s]

CO2 intensity 24 hr forecast MAPE: 6.59%


In [42]:
len(past_covariates_cols)

11

In [43]:
ablation_df = pd.DataFrame({'Past Covariate Dropped': ['co2_emissions_consumed', 'ng_ng',  'ng_wnd', 'ng_sun', 'ng_wat', 'ng_col', 'ng_nuc', 'd'],
                          'MAPE': mapes,
                          'Baseline MAPE': [co2_24_hist_fcst_mape_all_cov] * len(mapes)})
ablation_df['Change in MAPE'] = ablation_df['MAPE'] - ablation_df['Baseline MAPE']
ablation_df

,Past Covariate Dropped,MAPE,Baseline MAPE,Change in MAPE
0,co2_emissions_consumed,6.515598,6.56391,-0.048311
1,ng_ng,6.605591,6.56391,0.041681
2,ng_wnd,6.900029,6.56391,0.336120
3,ng_sun,6.748746,6.56391,0.184837
4,ng_wat,6.594273,6.56391,0.030363
5,ng_col,6.674869,6.56391,0.110959
6,ng_nuc,6.498367,6.56391,-0.065543
7,d,6.588889,6.56391,0.024980


## Remove one future covariate

In [47]:
future_covariates_cols = ['ng_ng_fcst', 'ng_col_fcst', 'ng_sun_fcst', 'ng_wat_fcst', 'ng_wnd_fcst']
future_covariates_drop_one = []

for i in range(len(future_covariates_cols)):
    future_covariates_drop_one.append(future_covariates_cols[:i] + future_covariates_cols[i+1:])

In [48]:
mapes = []

for future_covariates in future_covariates_drop_one:

    past_covariates_cols = ['co2_emissions_consumed', 'ng_ng',  'ng_wnd', 'ng_sun', 'ng_wat', 'ng_col', 'ng_nuc', 'd'] +\
                                        ['utc_month', 'utc_week_of_year', 'utc_day_of_week', 'utc_hour']

    future_covariates_cols = future_covariates + ['utc_month', 'utc_week_of_year', 'utc_day_of_week', 'utc_hour']

    if past_covariates_cols:
        past_covariates = TimeSeries.from_dataframe(df=df,
                          value_cols=past_covariates_cols,
                          freq='H')
    if future_covariates_cols:
        future_covariates = TimeSeries.from_dataframe(df=df,
                          value_cols=future_covariates_cols,
                          freq='H')

    lag_params = {
        'lags': LAGS,
        'lags_past_covariates': LAGS_PAST_COVARIATES,
        'lags_future_covariates': [1, 2, 3, 4, 5, 6, 12, 18, 24]
    }

    optional_covariates = {
        'past_covariates': past_covariates,
        'future_covariates': future_covariates,
    }

    optional_model_params = {
        'random_state': RANDOM_STATE#,
        #'likelihood': 'quantile',    # Uncomment these for probabilistic model
        #'quantiles': [0.05, 0.5, 0.95] # Middle value must be 0.5
    }

    optional_samples = {
        'num_samples': None # Set this to 'None' for deterministic, 100 or other for probabalistic
    }

    optional_hist_fcst_params = {
        'start': TRAIN_TEST_SPLIT_TIMESTAMP,
        'forecast_horizon': 24, # only a 24 hour forecast horizon
        'stride': STRIDE,
        'retrain': 730,
        'last_points_only': LAST_POINTS_ONLY,
        'verbose': VERBOSE
    }

    # drop keys which are None
    lag_params = {k: v for k, v in lag_params.items() if v is not None}
    optional_covariates = {k: v for k, v in optional_covariates.items() if v is not None}
    optional_model_params = {k: v for k, v in optional_model_params.items() if v is not None}
    optional_samples = {k: v for k, v in optional_samples.items() if v is not None}
    optional_hist_fcst_params = {k: v for k, v in optional_hist_fcst_params.items() if v is not None}

    co2_24_historical_fcsts, co2_24_hr_model = make_fit_hist_fcst_lgbm_model(
                                                                                        series=series,
                                                                                        lag_params=lag_params,
                                                                                        optional_model_params=optional_model_params,
                                                                                        optional_covariates=optional_covariates,
                                                                                        optional_hist_fcst_params=optional_hist_fcst_params
                                                                                )
    co2_24_hist_fcst_mape = calc_mape_from_hist_fcsts(val=test, historical_fcsts=co2_24_historical_fcsts)
    print(f'CO2 intensity 24 hr forecast MAPE: {round(co2_24_hist_fcst_mape, 2)}%')
    mapes.append(co2_24_hist_fcst_mape)

  0%|          | 0/368 [00:00<?, ?it/s]

CO2 intensity 24 hr forecast MAPE: 6.56%


  0%|          | 0/368 [00:00<?, ?it/s]

CO2 intensity 24 hr forecast MAPE: 6.66%


  0%|          | 0/368 [00:00<?, ?it/s]

CO2 intensity 24 hr forecast MAPE: 6.93%


  0%|          | 0/368 [00:00<?, ?it/s]

CO2 intensity 24 hr forecast MAPE: 6.43%


  0%|          | 0/368 [00:00<?, ?it/s]

CO2 intensity 24 hr forecast MAPE: 6.52%


In [49]:
ablation_df = pd.DataFrame({'Future Covariate Dropped': ['ng_ng_fcst', 'ng_col_fcst', 'ng_sun_fcst', 'ng_wat_fcst', 'ng_wnd_fcst'],
                          'MAPE': mapes,
                          'Baseline MAPE': [co2_24_hist_fcst_mape_all_cov] * len(mapes)})
ablation_df['Change in MAPE'] = ablation_df['MAPE'] - ablation_df['Baseline MAPE']
ablation_df

,Future Covariate Dropped,MAPE,Baseline MAPE,Change in MAPE
0,ng_ng_fcst,6.562399,6.56391,-0.001511
1,ng_col_fcst,6.660377,6.56391,0.096468
2,ng_sun_fcst,6.928496,6.56391,0.364586
3,ng_wat_fcst,6.428001,6.56391,-0.135908
4,ng_wnd_fcst,6.515090,6.56391,-0.048820
